In [1]:
from vllm import LLM
from tqdm import tqdm
from vllm.sampling_params import SamplingParams

from data_process import get_cxr_paths_list
import json
import ast

INFO 05-23 21:19:14 [__init__.py:239] Automatically detected platform cuda.


In [2]:
cxr_paths = get_cxr_paths_list('/home/than/DeepLearning/CheXzero/data/cxr_paths.csv')
txt_folder = '/home/than/physionet.org/files/mimic-cxr-jpg/2.0.0/reports/files/'

cxr_path = cxr_paths[2000]
tokens = cxr_path.split('/')
study_num = tokens[-2]
patient_num = tokens[-3]
patient_group = tokens[-4]
filename = study_num + '.txt'
txt_report = txt_folder + patient_group + '/' + patient_num + '/' + study_num + '.txt'

f = open(txt_report, 'r')
s = f.read()
s_split = s.split()
s = ' '.join(s_split)

print(s)

FINAL REPORT CHEST, TWO VIEWS: ___. HISTORY: ___-year-old male with new onset of confusion. FINDINGS: PA and lateral views of the chest. No prior. The lungs are clear. The costophrenic angles are sharp where seen. However, the right posterior costophrenic angle is not included in the field of view. The cardiomediastinal silhouette is within normal limits. Osseous and soft tissue structures are notable for degenerative changes of the right acromioclavicular joint and hypertrophic changes in the spine. IMPRESSION: No acute cardiopulmonary process.


In [ ]:
model_name = "mistralai/Ministral-8B-Instruct-2410"
sampling_params = SamplingParams(max_tokens=8192, temperature=0, top_k=-1)
llm = LLM(model=model_name, tokenizer_mode="mistral", config_format="mistral", load_format="mistral")

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

INFO 05-23 21:19:28 [config.py:689] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 05-23 21:19:29 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

INFO 05-23 21:19:32 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None, served_model_name=microsoft/phi-4, num_scheduler_steps=1, multi_step_stream_outputs=True, enable_prefix_caching=True, chunked_prefill_enabled=True, use_async_output_proc=True, disable_mm_pre

: 

In [5]:
prompt = f"""
You are a helpful assistant. Please respond in valid JSON only.
Question: What are the descriptive observations in the report? FINAL REPORT EXAMINATION: CHEST (PORTABLE AP)CHEST (PORTABLE AP)i INDICATION: ___ year old woman with SOB // eval for sign of PNA, effusion, pulm vascular congestion COMPARISON: Chest radiographs ___ IMPRESSION: Mild pulmonary edema and small to moderate bilateral pleural effusions all improved since ___ following extubation. Heart size normal. No pneumothorax. Left subclavian line ends in the SVC 
Respond with a JSON object with this structure:
{{
  "observations": ["Mild pulmonary edema", "Small to moderate bilateral pleural effusions", "Heart size normal", "No pneumothorax", "Left subclavian line ends in the SVC"],    
}}
Question: What are the descriptive observations in the report? FINAL REPORT EXAMINATION: CHEST (AP AND LAT) INDICATION: History: ___M with cough, fever TECHNIQUE: Upright AP and lateral views of the chest COMPARISON: Chest CT ___ FINDINGS: Cardiac silhouette size is normal. Mediastinal and hilar contours are unremarkable. Lungs are hyperinflated. No pulmonary edema is seen. Ill-defined patchy opacities are noted in the left lung base, concerning for pneumonia. Blunting of the costophrenic angles bilaterally suggests trace bilateral pleural effusions, more pronounced on the left. No pneumothorax is present. No acute osseous abnormalities detected. Multiple clips are again noted at the gastroesophageal junction and in the right upper quadrant of the abdomen. IMPRESSION: Patchy ill-defined left basilar opacity concerning for pneumonia. Small bilateral pleural effusions.
Respond with a JSON object with this structure:
{{
  "observations": ["Cardiac silhouette size is normal", "Mediastinal and hilar contours are unremarkable", "Lungs are hyperinflated", "No pulmonary edema is seen", "Ill-defined patchy opacities in the left lung base", "Blunting of the costophrenic angles bilaterally", "No pneumothorax", "No acute osseous abnormalities", "Small bilateral pleural effusions"],    
}}
Question: What are the descriptive observations in the report? {s}
Respond with a JSON object with this structure:
"""

In [6]:
messages = [
    {
        "role": "user",
        "content": prompt
    },
]

outputs = llm.chat(messages, 
                sampling_params=sampling_params)

result = {
    "id": 0,
    "model_output": outputs[0].outputs[0].text,
}

INFO 05-04 22:24:02 [chat_utils.py:396] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
WARNING 05-04 22:24:02 [chat_utils.py:317] 'add_generation_prompt' is not supported for mistral tokenizer, so it will be ignored.
WARNING 05-04 22:24:02 [chat_utils.py:321] 'continue_final_message' is not supported for mistral tokenizer, so it will be ignored.


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [7]:
print(result['model_output'])

import json
json.loads(result['model_output'])['observations']

{
  "observations": ["Lungs are clear", "Costophrenic angles are sharp where seen", "Right posterior costophrenic angle not included in the field of view", "Cardiomediastinal silhouette is within normal limits", "No acute cardiopulmonary process", "Degenerative changes of the right acromioclavicular joint", "Hypertrophic changes in the spine"]
}


['Lungs are clear',
 'Costophrenic angles are sharp where seen',
 'Right posterior costophrenic angle not included in the field of view',
 'Cardiomediastinal silhouette is within normal limits',
 'No acute cardiopulmonary process',
 'Degenerative changes of the right acromioclavicular joint',
 'Hypertrophic changes in the spine']

In [16]:
with open('/home/than/DeepLearning/CheXzero/data/mimic_concepts.json', 'r') as f:
    results = json.load(f)

concepts = []
for result in tqdm(results):
    try:
        re_dict = ast.literal_eval(result['model_output'])
        concepts.extend(re_dict['observations'])
    except:
        print(result['model_output'])
        continue

concepts = list(set(concepts))

  7%|▋         | 25508/377110 [00:00<00:04, 83270.47it/s]

{
  "observations": ["Interval enlargement of the right pleural effusion", "Pulmonary vascular congestion", "Left chest wall triple lead pacing device", "No left-sided effusion", "Linear opacity in the left lower lung", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", "No left-sided effusion", 

 18%|█▊        | 67882/377110 [00:00<00:03, 83883.23it/s]

{
  "observations": ["Pulmonary vascular and mediastinal venous distention have increased", "New mild pulmonary edema", "Increase in heart size", "Small left pleural effusion", "No pneumothorax", "ET tube at the thoracic inlet", "Nasogastric tube below the lower esophagus", "Right PIC line traced to low SVC", "Tip of the ET tube is at the thoracic inlet", "Nasogastric tube passes below the lower esophagus and out of view", "Right PIC line can be traced as far as the low SVC", "Tip of the ET tube is at the thoracic inlet", "Nasogastric tube passes below the lower esophagus and out of view", "Right PIC line can be traced as far as the low SVC", "Tip of the ET tube is at the thoracic inlet", "Nasogastric tube passes below the lower esophagus and out of view", "Right PIC line can be traced as far as the low SVC", "Tip of the ET tube is at the thoracic inlet", "Nasogastric tube passes below the lower esophagus and out of view", "Right PIC line can be traced as far as the low SVC", "Tip of t

 36%|███▌      | 134095/377110 [00:01<00:03, 80862.80it/s]

...

 64%|██████▍   | 241211/377110 [00:02<00:01, 82815.30it/s]

{
  "observations": ["Moderate cardiomegaly is chronic", "Pulmonary vasculature is normal", "Previous right pleural effusion has almost resolved", "Partial atelectasis in the right middle lobe", "Partial atelectasis in the right middle lobe has been recurrent problem", "Partial atelectasis in the right middle lobe has been recurrent problem, more severe in ___", "Partial atelectasis in the right middle lobe has been recurrent problem, more severe in ___", "Partial atelectasis in the right middle lobe has been recurrent problem, more severe in ___", "Partial atelectasis in the right middle lobe has been recurrent problem, more severe in ___", "Partial atelectasis in the right middle lobe has been recurrent problem, more severe in ___", "Partial atelectasis in the right middle lobe has been recurrent problem, more severe in ___", "Partial atelectasis in the right middle lobe has been recurrent problem, more severe in ___", "Partial atelectasis in the right middle lobe has been recurrent 

 84%|████████▎ | 315209/377110 [00:03<00:00, 80693.41it/s]

{
  "observations": ["Interval progression of previously visualized retrocardiac opacity", "New opacity silhouetting the right heart border", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneumothorax", "No evidence of pneum

100%|██████████| 377110/377110 [00:04<00:00, 81552.18it/s]


In [27]:
len(concepts)
concepts[:10]
# filter concepts that contain specific disease names, no matter letter case
filtered_concepts = list({concept for concept in concepts if any(disease.lower() not in concept.lower() for disease in ['pleural effusion'])})
filtered_concepts[:10]

['Right lower lobe pleural-based density previously described as rounded atelectasis',
 'Moderate left effusion which appears partially loculated',
 'Nasogastric tube courses into the stomach and out of view',
 'Increased lucency at the right CP angle and right apex',
 'Mild compression deformity of T6 vertebral body',
 'Cardiac silhouette is otherwise unremarkable',
 'Conspicuous retrocardiac opacities',
 'Small amount of right apical pneumothorax is persistent',
 'Chronic interstitial changes compatible with bronchiectasis',
 'Cardiac silhouette is normal in size and appears decreased compared to ___']